# ASG Quant Fund Project

#### Armaan Gandhara

## Week 1

### Data Loader Development

In [1]:
import yfinance as yf
import pandas as pd
from typing import List, Union

In [ ]:
class data_loader:

    def __init__(self):
        pass

    def get_data(self, ticker: str, start: str, end: str) -> pd.DataFrame:
        ticker = ticker.replace('.', '-')

        data = yf.download(ticker, start=start, end=end, progress=False)
        if data.empty:
            print(f"[!] Failed to download {ticker}. Skipping.")
            return None
        data.dropna(inplace=True)
        if 'Adj Close' in data.columns:
            data.drop(columns=['Adj Close'], inplace=True)
        #data.rename(columns={'Open': 'open','High': 'high','Low': 'low','Close': 'close','Volume': 'volume'}, inplace=True)

        #required_cols = ['open', 'high', 'low', 'close', 'volume']
        #data = data[required_cols]
        data = data.droplevel('Ticker', axis=1)
        data.reset_index(inplace=True)
        data.index = data['Date']
        del data['Date']
        data.index.name = 'Date'

        return data


    def get_multiple_data(self, tickers: List[str], start: str, end: str) -> dict:
        data_dict = {}
        for ticker in tickers:
            df = self.get_data(ticker, start, end)
            if df is not None:
                data_dict[ticker] = df

        return data_dict

In [18]:
dt = data_loader()
tsla = dt.get_data("TSLA", '2020-01-01', '2020-01-10')
tsla
tickers = ['AAPL', 'BRK.B', 'JPM']
dater = dt.get_multiple_data(tickers, '2025-01-01', '2025-01-10')
dater['BRK.B']

Price,Date,Close,High,Low,Open,Volume
0,2025-01-02,451.100006,456.890015,450.029999,455.959991,3746400
1,2025-01-03,453.559998,454.529999,450.119995,452.529999,2884600
2,2025-01-06,451.410004,456.239990,450.570007,453.850006,4072900
3,2025-01-07,452.920013,456.519989,451.100006,452.799988,3507200
4,2025-01-08,451.839996,454.000000,449.630005,453.630005,3933300


### Strategy 1 Mean Reversion

#### Not Used

In [9]:
import pandas as pd
import pandas_ta as ta

c:\Users\agand\OneDrive\Desktop\ASG MICROFUND\venv\Lib\site-packages\pandas_ta\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [ ]:

class mean_reversion_strategy:
    def __init__(self, lookback: int = 20, std_dev: float = 2.0, threshold: float = 0.0):
        self.lookback = lookback
        self.std_dev = std_dev
        self.threshold = threshold

    def generate_signals(self, data: pd.DataFrame) -> pd.DataFrame:
        df = data.copy()

        bb = ta.bbands(close=df['Close'], length=self.lookback, std=self.std_dev)

        if bb is None or bb.empty:
            raise ValueError("Bollinger Bands calculation failed. Check input data.")

        df = df.join(bb)

        df['signal'] = 0  # Default to hold (0)
        
        df.loc[df['Close'] < df[f'BBL_{self.lookback}_{self.std_dev}'] * (1 - self.threshold), 'signal'] = 1
        
        df.loc[df['Close'] > df[f'BBU_{self.lookback}_{self.std_dev}'] * (1 + self.threshold), 'signal'] = -1

        return df[['signal']]


In [48]:
dt = data_loader()
tsla = dt.get_data("TSLA", '2025-01-01', '2025-06-01')
mrs = mean_reversion_strategy()
signals = mrs.generate_signals(tsla)
signals.value_counts()


signal
 0        92
 1         7
-1         3
Name: count, dtype: int64

#### Used

In [ ]:
from backtesting import Strategy, Backtest
import pandas as pd
import pandas_ta as pdt

class mean_reversion_strategy(Strategy):
    length = 20
    std = 2.0
    def init(self):
        price = pd.Series(self.data.Close)
        bb = ta.bbands(close=price, length=self.length, std=self.std)

        self.lower = self.I(lambda: bb[f'BBL_{self.length}_{self.std}'])
        self.upper = self.I(lambda: bb[f'BBU_{self.length}_{self.std}'])

    def next(self):
        price = self.data.Close[-1]
        if price < self.lower[-1] and not self.position:
            self.buy(size=int(self.equity / price))
        elif price > self.upper[-1] and not self.position:
            self.sell(size=int(self.equity / price))

        if self.position.is_long and price > self.data.Close[-2]:
            self.position.close()
        elif self.position.is_short and price < self.data.Close[-2]:
            self.position.close()



### Backtesting Engine

In [10]:
class GenericBacktestEngine:
    def __init__(self, data: pd.DataFrame, strategy_cls, strategy_kwargs: dict = None, cash: float = 10000, commission: float = 0.002):
        self.data = data
        self.strategy_cls = strategy_cls
        self.strategy_kwargs = strategy_kwargs or {}
        self.cash = cash
        self.commission = commission
        

    def run(self):
        bt = Backtest(
            self.data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        stats = bt.run(**self.strategy_kwargs)
        return stats

    def plot(self):
        bt = Backtest(
            self.data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        bt.run(**self.strategy_kwargs)
        bt.plot()


In [17]:
dt = data_loader()
tsla = dt.get_data("TSLA", '2020-01-01', '2025-06-01')
bt = Backtest(tsla, mean_reversion_strategy, cash=10000, commission=0.002)
stats = bt.run()
bt.plot()



C:\Users\agand\AppData\Local\Temp\ipykernel_5112\1597647998.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(tsla, mean_reversion_strategy, cash=10000, commission=0.002)


GridPlot(id='p3391', ...)

### Improving the Mean Reversion Strategy

In [121]:
from backtesting import Strategy, Backtest
class mean_reversion_strategy(Strategy):
    length = 20
    std = 2.0
    
    def init(self):
        price = pd.Series(self.data.Close)
        bb = ta.bbands(close=price, length=self.length, std=self.std)
        self.lower = self.I(lambda: bb[f'BBL_{self.length}_{self.std}'])
        self.upper = self.I(lambda: bb[f'BBU_{self.length}_{self.std}'])

    def next(self):
        price = self.data.Open[-1]
        # Entry
        if price < self.lower[-1]*1.2 and not self.position:
            self.buy(size=int(self.equity / price))
        elif price > self.upper[-1] and not self.position:
            self.sell(size=int(self.equity / price))

        # Exit with 2% tolerance
        if self.position.is_long and price >= self.upper[-1]:
            self.position.close()
        elif self.position.is_short and price <= self.lower[-1]:
            self.position.close()

In [124]:
dt = data_loader()
data = dt.get_data("TSLA", '2023-01-01', '2025-06-01')

#Fix index issues
data.index = data['Date']
del data['Date']
data.index.name = 'Date'

engine = GenericBacktestEngine(
    data=data,
    strategy_cls=mean_reversion_strategy,
    strategy_kwargs={'length': 20, 'std': 2.0},
    cash=10000,
    commission=0.000
)

results = engine.run()
engine.plot()
print(results)

Start                     2023-01-03 00:00:00
End                       2025-05-30 00:00:00
Duration                    878 days 00:00:00
Exposure Time [%]                    89.40397
Equity Final [$]                  21520.96101
Equity Peak [$]                   24802.92055
Return [%]                          115.20961
Buy & Hold Return [%]               100.01154
Return (Ann.) [%]                    37.68165
Volatility (Ann.) [%]                93.07515
CAGR [%]                             24.60533
Sharpe Ratio                          0.40485
Sortino Ratio                         0.98371
Calmar Ratio                          0.85242
Alpha [%]                            85.52251
Beta                                  0.29684
Max. Drawdown [%]                   -44.20528
Avg. Drawdown [%]                    -9.83919
Max. Drawdown Duration      314 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   14
Win Rate [%]                      